<a href="https://colab.research.google.com/github/Venu0046/Gen-AI-Project/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, metrics
import matplotlib.pyplot as plt


In [10]:
# Load Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

# Normalize the images to the range [0, 1] and expand dimensions for channels
x_train = np.expand_dims(x_train.astype('float32') / 255.0, -1)
x_test = np.expand_dims(x_test.astype('float32') / 255.0, -1)


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding an item of clothing."""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.random.normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


In [12]:
# Encoder
encoder_input = layers.Input(shape=(28, 28, 1), name="encoder_input")
x = layers.Conv2D(32, (3, 3), strides=2, activation="relu", padding="same")(encoder_input)
x = layers.Conv2D(64, (3, 3), strides=2, activation="relu", padding="same")(x)
x = layers.Conv2D(128, (3, 3), strides=2, activation="relu", padding="same")(x)
shape_before_flattening = tf.keras.backend.int_shape(x)[1:]
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
z_mean = layers.Dense(2, name="z_mean")(x)
z_log_var = layers.Dense(2, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = models.Model(encoder_input, [z_mean, z_log_var, z], name="encoder")


In [13]:
# Decoder
decoder_input = layers.Input(shape=(2,), name="decoder_input")
x = layers.Dense(7 * 7 * 64)(decoder_input)  # Use 7*7*64 to match the shape_before_flattening
x = layers.Reshape((7, 7, 64))(x)
x = layers.Conv2DTranspose(64, (3, 3), strides=2, activation='relu', padding="same")(x)  # 14x14x64
x = layers.Conv2DTranspose(32, (3, 3), strides=2, activation='relu', padding="same")(x)  # 28x28x32
decoder_output = layers.Conv2D(1, (3, 3), activation="sigmoid", padding="same", name="decoder_output")(x)  # Output 28x28x1
decoder = models.Model(decoder_input, decoder_output)


In [14]:
class KLLossLayer(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
        return kl_loss


In [15]:
# VAE model
class VAE(models.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = metrics.Mean(name="reconstruction_loss")
        self.kl_loss_tracker = metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstruction = self.decoder(z)
        return z_mean, z_log_var, reconstruction

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, reconstruction = self(data)
            reconstruction_loss = tf.reduce_mean(
                tf.keras.losses.binary_crossentropy(data, reconstruction)
            )
            kl_loss = tf.reduce_mean(
                -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1)
            )
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {m.name: m.result() for m in self.metrics}

vae = VAE(encoder, decoder)
vae.compile(optimizer="adam")


In [ ]:
# Training
vae.fit(x_train,epochs=5,batch_size=64)


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 133s 139ms/step - kl_loss: 7.8556e-05 - reconstruction_loss: 0.5202 - total_loss: 0.5203
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 141s 137ms/step - kl_loss: 6.1146e-06 - reconstruction_loss: 0.4911 - total_loss: 0.4911
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 142s 138ms/step - kl_loss: 3.1021e-06 - reconstruction_loss: 0.4911 - total_loss: 0.4911
Epoch 4/5
125/938 ━━━━━━━━━━━━━━━━━━━━ 1:52 139ms/step - kl_loss: 3.4070e-06 - reconstruction_loss: 0.4917 - total_loss: 0.4917

In [ ]:
# Evaluation
example_images = x_test[:10]  # Reduced number of examples for simplicity
predictions = vae.predict(example_images)

# Plot examples
plt.figure(figsize=(10, 5))
for i in range(10):
    plt.subplot(2, 10, i + 1)
    plt.imshow(np.squeeze(example_images[i]), cmap='gray')
    plt.axis('off')
    plt.subplot(2, 10, 10 + i + 1)
    plt.imshow(np.squeeze(predictions[i]), cmap='gray')
    plt.axis('off')
plt.show()
